In [ ]:
!pip3 install pulp


In [5]:
from pulp import *

# Here is a useful function to implement the LHS upper bound that we need for the encoding
def lhsUpperBound(c_list, bounds):
    n = len(c_list)
    assert len(bounds) == n 
    upper_bnd = sum([(cj*lj) if cj < 0 else cj*uj for (cj, (lj, uj)) in zip(c_list, bounds) ])
    return upper_bnd

def solveForMaximumInequalitySatisfaction(n, m, c_matrix, d_values, bounds):
    # always check pre-conditions: saves so much time later
    assert len(c_matrix) == m
    assert all(len(c_list) == n for c_list in c_matrix)
    assert len(d_values) == m
    assert len(bounds) == n
    assert all (lj <= uj for (lj, uj) in bounds)
    
    
    prob = LpProblem("MaximizeInequalitiesSatisfied", LpMaximize)
    
    # Define decision variables
    x = []
    for i in range(n):
        x.append(LpVariable(f"x{i}", bounds[i][0],bounds[i][1]))
    
    satisfied = []
    for j in range(m):
        satisfied.append(LpVariable(f"satisfy{j}", 0, 1, cat="Binary"))
        
    prob += lpSum(satisfied)
    
    # Define Constraints
    for j in range(m):
        lhs = lpSum([c_matrix[j][i] * x[i] for i in range(n)])
        prob += (lhs <= d_values[j] + (1 - satisfied[j]) * 1e6)  
        prob += (lhs >= d_values[j] - 1e6 * satisfied[j]) 
    
    
    # Solve the problem
    prob.solve()
    
    # Extract results
    k = sum(1 for j in range(m) if satisfied[j].varValue > 0.5) 
    x_values = [x[i].varValue for i in range(n)] 
    
    return k, x_values
    

In [6]:
def testSolution(n, m, c_matrix, d_values, bounds, x_values, k_expected):
    # always check pre-conditions: saves so much time later
    assert len(c_matrix) == m
    assert all(len(c_list) == n for c_list in c_matrix)
    assert len(d_values) == m
    assert len(bounds) == n
    assert all (lj <= uj for (lj, uj) in bounds)
    assert len(x_values) == n
    assert 0 <= k_expected <= m
    # check solution within bounds
    for i in range(n):
        (lb, ub) = bounds[i]
        assert lb <= x_values[i] <= ub, f'x_{i} fails to be within its bounds {[lb, ub]}'
    # Check how many inequalities satisfied
    num_ineqs = 0
    
    for (c_list, d) in zip(c_matrix, d_values):
        if sum([cj * xj for (cj, xj) in zip(c_list,x_values )]) <= d+1E-3:
            num_ineqs = num_ineqs + 1
    assert num_ineqs == k_expected, f' Expected number of inequalities to be sat: {k_expected} your solution satisfies: {num_ineqs} inequalities '
    print('Test Passed')
    return 
        
        
        
        

# x_1 - x_2 & \leq -5 & \leftarrow I_1 \\ 
# x_1 + 2 x_2 & \leq 3 & \leftarrow I_2 \\ 
# x_1 & \geq 4  & \leftarrow I_3\\ 
# x_1 & \leq -2  & \leftarrow I_4\\ 
# x_2 & \geq 3  & \leftarrow I_5\\ 
# x_2 & \leq -1  & \leftarrow I_6\\ 

n = 2
m = 6
c_matrix = [
    [1, -1],
    [1, 2],
    [-1, 0],
    [1, 0],
    [-1, 0],
    [1, 0]
]

d_list = [
    -5, 3, -4, -2, -3, -1
]

bounds = [(-10, 10), (-10, 10)]

(k, x_values) = solveForMaximumInequalitySatisfaction(n, m, c_matrix, d_list, bounds)
testSolution(n, m, c_matrix, d_list, bounds, x_values, 4)
print('5 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josiahroa/workspace/cs-masters/5414-greedy-algorithms/karatsuba/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/aa6e16e125064e85ab3fe07097628333-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/aa6e16e125064e85ab3fe07097628333-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 64 RHS
At line 77 BOUNDS
At line 88 ENDATA
Problem MODEL has 12 rows, 8 columns and 28 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5.99999 - 0.00 seconds
Cgl0004I processed model has 6 rows, 8 columns (6 integer (6 of which binary)) and 14 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 1.15e-05